In [1]:
import numpy as np
import pandas as pd
from gurobipy import GRB
import gurobipy as gp
import random
import networkx as nx
import math

In [2]:
NodeCount = 20
GraphDensityCoefficient = 0.5
MaxCostArc = 20
RequiredInfo = 40
CommodityCount = 5
MaxDemandCommodity = 20
MaxCapacityArc = 80

random.seed(10)

N = [i+1 for i in range(NodeCount)]
Commodities = [i+1 for i in range(CommodityCount)]
NodeNodeAdjacencyMatrix = np.zeros((NodeCount,NodeCount))


ArcProbability = np.zeros((NodeCount,NodeCount))

for i in range(NodeCount-1):
    for j in range(i+1,NodeCount):
        ArcProbability[i,j] = random.uniform(0,1)
        if ArcProbability[i,j] > GraphDensityCoefficient:
            
            NodeNodeAdjacencyMatrix[i,j] = 1
            
    if sum(NodeNodeAdjacencyMatrix[i]) == 0:
        
        NodeNodeAdjacencyMatrix[i,NodeCount-1] = 1

ArcCount = int(sum(sum(NodeNodeAdjacencyMatrix)))



In [3]:
NodeNodeAdjacencyMatrix

array([[0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1.,
        1., 0., 0., 1.],
       [0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0.,
        1., 0., 1., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0.,
        1., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
        0., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1.,
        1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
        1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1.,
        0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0.,
        1., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1.,
        0., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0.

In [4]:
ArcCount    

93

In [5]:
# G = nx.from_numpy_array(NodeNodeAdjacencyMatrix,create_using=nx.DiGraph())
# nx.draw_circular(G,arrows=True)

In [6]:
NodeNodeAdjacencyList = {i:[] for i in N}
CostDict = {}
CapacityDict = {}
ArcDict = {i:0 for i in range(ArcCount)}
InfoDict = {i:random.randint(0,RequiredInfo/2) for i in N}

ArcCounter = 0

for i in range(NodeCount):
    for j in range(NodeCount):
        if NodeNodeAdjacencyMatrix[i][j] == 1:
            NodeNodeAdjacencyList[i+1].append(j+1)
            CostDict[ArcCounter]= random.randint(0,MaxCostArc)
            CapacityDict[ArcCounter] = random.randint(MaxCapacityArc/2,MaxCapacityArc)
            ArcDict[ArcCounter] = (i+1,j+1)
            ArcCounter += 1

In [7]:
ArcDict

{0: (1, 2),
 1: (1, 4),
 2: (1, 6),
 3: (1, 7),
 4: (1, 8),
 5: (1, 10),
 6: (1, 13),
 7: (1, 14),
 8: (1, 16),
 9: (1, 17),
 10: (1, 20),
 11: (2, 4),
 12: (2, 5),
 13: (2, 7),
 14: (2, 8),
 15: (2, 9),
 16: (2, 10),
 17: (2, 14),
 18: (2, 17),
 19: (2, 19),
 20: (3, 5),
 21: (3, 6),
 22: (3, 12),
 23: (3, 13),
 24: (3, 14),
 25: (3, 15),
 26: (3, 17),
 27: (3, 19),
 28: (4, 5),
 29: (4, 8),
 30: (4, 9),
 31: (4, 14),
 32: (4, 19),
 33: (4, 20),
 34: (5, 6),
 35: (5, 11),
 36: (5, 16),
 37: (5, 18),
 38: (5, 19),
 39: (6, 7),
 40: (6, 8),
 41: (6, 9),
 42: (6, 10),
 43: (6, 16),
 44: (6, 17),
 45: (7, 8),
 46: (7, 12),
 47: (7, 17),
 48: (8, 9),
 49: (8, 11),
 50: (8, 12),
 51: (8, 13),
 52: (8, 15),
 53: (8, 16),
 54: (8, 19),
 55: (8, 20),
 56: (9, 10),
 57: (9, 11),
 58: (9, 12),
 59: (9, 14),
 60: (9, 15),
 61: (9, 17),
 62: (9, 18),
 63: (9, 19),
 64: (10, 11),
 65: (10, 12),
 66: (10, 13),
 67: (10, 14),
 68: (10, 16),
 69: (10, 18),
 70: (10, 19),
 71: (11, 17),
 72: (11, 18),


In [8]:
NodeNodeAdjacencyList

{1: [2, 4, 6, 7, 8, 10, 13, 14, 16, 17, 20],
 2: [4, 5, 7, 8, 9, 10, 14, 17, 19],
 3: [5, 6, 12, 13, 14, 15, 17, 19],
 4: [5, 8, 9, 14, 19, 20],
 5: [6, 11, 16, 18, 19],
 6: [7, 8, 9, 10, 16, 17],
 7: [8, 12, 17],
 8: [9, 11, 12, 13, 15, 16, 19, 20],
 9: [10, 11, 12, 14, 15, 17, 18, 19],
 10: [11, 12, 13, 14, 16, 18, 19],
 11: [17, 18, 20],
 12: [13, 14, 16, 17, 20],
 13: [14, 16, 17, 18, 19],
 14: [20],
 15: [16, 18, 19, 20],
 16: [20],
 17: [20],
 18: [20],
 19: [20],
 20: []}

In [9]:
# InfoList = [random.randint(0,MaxInfoNode) for i in N]
InfoDict

{1: 12,
 2: 5,
 3: 19,
 4: 13,
 5: 18,
 6: 2,
 7: 19,
 8: 9,
 9: 0,
 10: 18,
 11: 10,
 12: 20,
 13: 10,
 14: 2,
 15: 12,
 16: 19,
 17: 11,
 18: 9,
 19: 19,
 20: 5}

In [10]:
CostDict

{0: 7,
 1: 9,
 2: 17,
 3: 7,
 4: 2,
 5: 6,
 6: 18,
 7: 18,
 8: 12,
 9: 10,
 10: 6,
 11: 18,
 12: 1,
 13: 1,
 14: 18,
 15: 12,
 16: 19,
 17: 5,
 18: 3,
 19: 11,
 20: 0,
 21: 16,
 22: 2,
 23: 14,
 24: 11,
 25: 4,
 26: 15,
 27: 14,
 28: 19,
 29: 11,
 30: 10,
 31: 16,
 32: 12,
 33: 3,
 34: 20,
 35: 1,
 36: 20,
 37: 6,
 38: 13,
 39: 17,
 40: 1,
 41: 1,
 42: 9,
 43: 1,
 44: 5,
 45: 20,
 46: 11,
 47: 9,
 48: 12,
 49: 13,
 50: 11,
 51: 15,
 52: 5,
 53: 14,
 54: 1,
 55: 5,
 56: 13,
 57: 16,
 58: 3,
 59: 3,
 60: 3,
 61: 20,
 62: 9,
 63: 20,
 64: 11,
 65: 19,
 66: 19,
 67: 17,
 68: 0,
 69: 12,
 70: 3,
 71: 20,
 72: 3,
 73: 12,
 74: 6,
 75: 11,
 76: 6,
 77: 17,
 78: 0,
 79: 12,
 80: 0,
 81: 10,
 82: 1,
 83: 10,
 84: 15,
 85: 0,
 86: 4,
 87: 6,
 88: 7,
 89: 19,
 90: 18,
 91: 13,
 92: 0}

In [11]:
CapacityDict

{0: 50,
 1: 46,
 2: 43,
 3: 76,
 4: 66,
 5: 76,
 6: 68,
 7: 41,
 8: 52,
 9: 76,
 10: 73,
 11: 60,
 12: 77,
 13: 42,
 14: 63,
 15: 60,
 16: 80,
 17: 50,
 18: 78,
 19: 50,
 20: 74,
 21: 62,
 22: 65,
 23: 76,
 24: 43,
 25: 68,
 26: 79,
 27: 68,
 28: 51,
 29: 67,
 30: 75,
 31: 69,
 32: 52,
 33: 67,
 34: 49,
 35: 51,
 36: 80,
 37: 76,
 38: 63,
 39: 64,
 40: 78,
 41: 50,
 42: 54,
 43: 77,
 44: 46,
 45: 76,
 46: 51,
 47: 42,
 48: 78,
 49: 44,
 50: 52,
 51: 50,
 52: 65,
 53: 61,
 54: 51,
 55: 77,
 56: 49,
 57: 65,
 58: 65,
 59: 55,
 60: 67,
 61: 59,
 62: 74,
 63: 46,
 64: 73,
 65: 70,
 66: 55,
 67: 58,
 68: 50,
 69: 54,
 70: 72,
 71: 70,
 72: 76,
 73: 77,
 74: 58,
 75: 44,
 76: 79,
 77: 73,
 78: 44,
 79: 42,
 80: 67,
 81: 51,
 82: 62,
 83: 80,
 84: 50,
 85: 53,
 86: 58,
 87: 55,
 88: 64,
 89: 74,
 90: 78,
 91: 75,
 92: 48}

In [12]:
B_k = [random.randint(1,MaxDemandCommodity) for i in range(CommodityCount)]
B_k

[1, 4, 1, 2, 6]

In [13]:
Origin = [random.randint(1,math.floor(NodeCount/2)) for i in range(CommodityCount)]
Destination = [random.randint(math.floor(NodeCount/2)+1,NodeCount) for i in range(CommodityCount)]

In [14]:
Origin

[2, 7, 2, 1, 7]

In [15]:
Destination

[13, 20, 17, 11, 19]

In [16]:
def Reachable(visited, graph, node): 
    visited.append(node)
    queue.append(node)
    Reachable = []

    while queue:          
        m = queue.pop(0) 
#         print (m, end = " ") 
        Reachable.append(m)

        for neighbour in graph[m]:
            if neighbour not in visited:
                visited.append(neighbour)
                queue.append(neighbour)
    return Reachable

In [17]:
for i in range(len(Origin)):
    visited = [] # List for visited nodes.
    queue = []     #Initialize a queue
    ReachableList = Reachable(visited,NodeNodeAdjacencyList,Origin[i])
#     print(i,Origin[i],Destination[i],ReachableList)
    k = len(ReachableList)

# #             break
    if Destination[i] not in ReachableList:
        for j in range(k):
            if ReachableList[j]>Origin[i]:
                Destination[i] = ReachableList[j]
                
        

In [18]:
Destination

[13, 20, 17, 11, 19]

In [19]:
B_k_Dict = {i:[] for i in Commodities}
Origin_Dict = {i:[] for i in Commodities}
Destination_Dict = {i:[] for i in Commodities}
for i in range(CommodityCount):
    B_k_Dict[i+1]=B_k[i]
    Origin_Dict[i+1] = Origin[i]
    Destination_Dict[i+1] = Destination[i]

In [20]:
NodeNodeAdjacencyListWithDummyArcs = {i:[] for i in N}
# PathCost = {}
# CapacityListInitial = {}
InitialHighCost = MaxCostArc*ArcCount + 1
InitialHighCapacity = MaxDemandCommodity*ArcCount + 1

for i in range(len(Origin)):

    NodeNodeAdjacencyListWithDummyArcs[Origin[i]].append(Destination[i])

NodeNodeAdjacencyListWithDummyArcs

{1: [11],
 2: [13, 17],
 3: [],
 4: [],
 5: [],
 6: [],
 7: [20, 19],
 8: [],
 9: [],
 10: [],
 11: [],
 12: [],
 13: [],
 14: [],
 15: [],
 16: [],
 17: [],
 18: [],
 19: [],
 20: []}

In [21]:
PathSetForCommodity = {i:[] for i in Commodities}
Paths = [i for i in range(CommodityCount)]

for i in range(CommodityCount):
    PathSetForCommodity[i+1].append(Paths[i])
PathSetForCommodity

{1: [0], 2: [1], 3: [2], 4: [3], 5: [4]}

In [22]:
Paths

[0, 1, 2, 3, 4]

In [23]:
PathDict = {i:[] for i in Paths}
NodesInPath = {i:[] for i in Paths}
for i in Paths:
    PathDict[i].append((Origin[i],Destination[i]))
    NodesInPath[i].append(Origin[i])
    NodesInPath[i].append(Destination[i])
PathDict

{0: [(2, 13)], 1: [(7, 20)], 2: [(2, 17)], 3: [(1, 11)], 4: [(7, 19)]}

In [24]:
NodesInPath

{0: [2, 13], 1: [7, 20], 2: [2, 17], 3: [1, 11], 4: [7, 19]}

In [25]:
PathCost = {p:InitialHighCost for p in Paths}
PathInfo = {p:0 for p in Paths}
for i in Paths:
    for k in NodesInPath[i]:
        PathInfo[i] += InfoDict[k]
    for j in range(ArcCount):
        if list(ArcDict.values())[j] == (PathDict[i][0]):
            ArcIndex = list(ArcDict.values())[j]
            PathCost[i] = CostDict[j]
for i in Paths:
    if PathCost[i] == InitialHighCost:
        PathInfo[i] = RequiredInfo

        
#         else:
#             PathCost[i] = InitialHighCost
    
Counter = 0

for i in N:
    for j in N:
        if (i in Origin) and (j in Destination):
            
            if NodeNodeAdjacencyMatrix[i-1,j-1] == 0:
                CapacityDict[ArcCount+Counter] = InitialHighCapacity
                CostDict[ArcCount+Counter] = InitialHighCost
                NodeNodeAdjacencyMatrix[i-1,j-1] = 1
                ArcDict[ArcCount+Counter] = (i,j)
                
                Counter += 1
                
#             else:
#                 PathCost[(i,j)] = CostList[(i,j)]
ArcCountWithDummyArcs = ArcCount + Counter

In [26]:
PathInfo

{0: 40, 1: 40, 2: 16, 3: 40, 4: 40}

In [27]:
PathCost

{0: 1861, 1: 1861, 2: 3, 3: 1861, 4: 1861}

In [28]:
CapacityDict

{0: 50,
 1: 46,
 2: 43,
 3: 76,
 4: 66,
 5: 76,
 6: 68,
 7: 41,
 8: 52,
 9: 76,
 10: 73,
 11: 60,
 12: 77,
 13: 42,
 14: 63,
 15: 60,
 16: 80,
 17: 50,
 18: 78,
 19: 50,
 20: 74,
 21: 62,
 22: 65,
 23: 76,
 24: 43,
 25: 68,
 26: 79,
 27: 68,
 28: 51,
 29: 67,
 30: 75,
 31: 69,
 32: 52,
 33: 67,
 34: 49,
 35: 51,
 36: 80,
 37: 76,
 38: 63,
 39: 64,
 40: 78,
 41: 50,
 42: 54,
 43: 77,
 44: 46,
 45: 76,
 46: 51,
 47: 42,
 48: 78,
 49: 44,
 50: 52,
 51: 50,
 52: 65,
 53: 61,
 54: 51,
 55: 77,
 56: 49,
 57: 65,
 58: 65,
 59: 55,
 60: 67,
 61: 59,
 62: 74,
 63: 46,
 64: 73,
 65: 70,
 66: 55,
 67: 58,
 68: 50,
 69: 54,
 70: 72,
 71: 70,
 72: 76,
 73: 77,
 74: 58,
 75: 44,
 76: 79,
 77: 73,
 78: 44,
 79: 42,
 80: 67,
 81: 51,
 82: 62,
 83: 80,
 84: 50,
 85: 53,
 86: 58,
 87: 55,
 88: 64,
 89: 74,
 90: 78,
 91: 75,
 92: 48,
 93: 1861,
 94: 1861,
 95: 1861,
 96: 1861,
 97: 1861,
 98: 1861,
 99: 1861,
 100: 1861,
 101: 1861}

In [29]:
ArcDict

{0: (1, 2),
 1: (1, 4),
 2: (1, 6),
 3: (1, 7),
 4: (1, 8),
 5: (1, 10),
 6: (1, 13),
 7: (1, 14),
 8: (1, 16),
 9: (1, 17),
 10: (1, 20),
 11: (2, 4),
 12: (2, 5),
 13: (2, 7),
 14: (2, 8),
 15: (2, 9),
 16: (2, 10),
 17: (2, 14),
 18: (2, 17),
 19: (2, 19),
 20: (3, 5),
 21: (3, 6),
 22: (3, 12),
 23: (3, 13),
 24: (3, 14),
 25: (3, 15),
 26: (3, 17),
 27: (3, 19),
 28: (4, 5),
 29: (4, 8),
 30: (4, 9),
 31: (4, 14),
 32: (4, 19),
 33: (4, 20),
 34: (5, 6),
 35: (5, 11),
 36: (5, 16),
 37: (5, 18),
 38: (5, 19),
 39: (6, 7),
 40: (6, 8),
 41: (6, 9),
 42: (6, 10),
 43: (6, 16),
 44: (6, 17),
 45: (7, 8),
 46: (7, 12),
 47: (7, 17),
 48: (8, 9),
 49: (8, 11),
 50: (8, 12),
 51: (8, 13),
 52: (8, 15),
 53: (8, 16),
 54: (8, 19),
 55: (8, 20),
 56: (9, 10),
 57: (9, 11),
 58: (9, 12),
 59: (9, 14),
 60: (9, 15),
 61: (9, 17),
 62: (9, 18),
 63: (9, 19),
 64: (10, 11),
 65: (10, 12),
 66: (10, 13),
 67: (10, 14),
 68: (10, 16),
 69: (10, 18),
 70: (10, 19),
 71: (11, 17),
 72: (11, 18),


In [30]:
Delta_ij_p = {i:[0]*ArcCountWithDummyArcs for i in Paths}

for p in Paths:
    for i in ArcDict:    
            if ArcDict[i] in PathDict[p]:
                Delta_ij_p[p][i] = 1
Delta_ij_p

{0: [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0],
 1: [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0

In [31]:
NodeNodeAdjacencyMatrix

array([[0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1.,
        1., 0., 1., 1.],
       [0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0.,
        1., 0., 1., 1.],
       [0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0.,
        1., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
        0., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1.,
        1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0.,
        1., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1.,
        0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0.,
        1., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1.,
        0., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0.

In [32]:
ReversedArcDict = {ArcDict[i]:i for i in range(ArcCountWithDummyArcs)}
ReversedArcDict

{(1, 2): 0,
 (1, 4): 1,
 (1, 6): 2,
 (1, 7): 3,
 (1, 8): 4,
 (1, 10): 5,
 (1, 13): 6,
 (1, 14): 7,
 (1, 16): 8,
 (1, 17): 9,
 (1, 20): 10,
 (2, 4): 11,
 (2, 5): 12,
 (2, 7): 13,
 (2, 8): 14,
 (2, 9): 15,
 (2, 10): 16,
 (2, 14): 17,
 (2, 17): 18,
 (2, 19): 19,
 (3, 5): 20,
 (3, 6): 21,
 (3, 12): 22,
 (3, 13): 23,
 (3, 14): 24,
 (3, 15): 25,
 (3, 17): 26,
 (3, 19): 27,
 (4, 5): 28,
 (4, 8): 29,
 (4, 9): 30,
 (4, 14): 31,
 (4, 19): 32,
 (4, 20): 33,
 (5, 6): 34,
 (5, 11): 35,
 (5, 16): 36,
 (5, 18): 37,
 (5, 19): 38,
 (6, 7): 39,
 (6, 8): 40,
 (6, 9): 41,
 (6, 10): 42,
 (6, 16): 43,
 (6, 17): 44,
 (7, 8): 45,
 (7, 12): 46,
 (7, 17): 47,
 (8, 9): 48,
 (8, 11): 49,
 (8, 12): 50,
 (8, 13): 51,
 (8, 15): 52,
 (8, 16): 53,
 (8, 19): 54,
 (8, 20): 55,
 (9, 10): 56,
 (9, 11): 57,
 (9, 12): 58,
 (9, 14): 59,
 (9, 15): 60,
 (9, 17): 61,
 (9, 18): 62,
 (9, 19): 63,
 (10, 11): 64,
 (10, 12): 65,
 (10, 13): 66,
 (10, 14): 67,
 (10, 16): 68,
 (10, 18): 69,
 (10, 19): 70,
 (11, 17): 71,
 (11, 18): 72,


In [33]:
def PathModel():
    m1 = gp.Model("PathModel")
    
    Flow = m1.addVars(Paths,Commodities,name="flow")

    NetworkCost = 0

    for k in Commodities:
        for p in PathSetForCommodity[k]:
            NetworkCost += B_k_Dict[k] * PathCost[p] * Flow[p,k]
        for p in Paths:
            if p not in PathSetForCommodity[k]:
                m1.addConstr(Flow[p,k]==0)
    
    m1.setObjective(NetworkCost, GRB.MINIMIZE)

    ArcFlow = [0]*ArcCountWithDummyArcs
    InfoCollected = 0

    for k in Commodities:
        for p in PathSetForCommodity[k]:
            for i in range(len(Delta_ij_p[p])):
                if Delta_ij_p[p][i] == 1:
                    ArcFlow[i] += B_k_Dict[k] * Flow[p,k]
            InfoCollected += PathInfo[p]* Flow[p,k]
    
    CapacityConstraint = [0]*ArcCountWithDummyArcs
    
#     for i in range(ArcCountWithDummyArcs):
    CapacityConstraint = m1.addConstrs((ArcFlow[i] <= CapacityDict[i] for i in range(ArcCountWithDummyArcs)),name="Cap")
        
    InfoThresholdConstraint = m1.addConstr(InfoCollected>=RequiredInfo,name="InfoT")
    
    FlowConstraint = m1.addConstrs(((Flow.sum('*',k))==1 for k in Commodities),name="FlowPart")
            
    m1.write("C:\PhD IE\Optimization Models for Large Networks\path.lp")
#     print(InfoThresholdConstraint)
    m1.optimize()
    
    if m1.Status == GRB.OPTIMAL:
        OptimalFlow = m1.getAttr('X',Flow)
#         ReducedCosts = m1.getAttr('RC',Flow)
#         print(ReducedCosts)
        Pi_ij = m1.getAttr('Pi',CapacityConstraint)
        ThresholdDual = m1.getConstrByName("InfoT")
        W = ThresholdDual.Pi
        Sigma = m1.getAttr('Pi',FlowConstraint)
        for k in Commodities:
            for p in PathSetForCommodity[k]:
                if OptimalFlow[p,k] != 0:
                    print("Flow on path",p,"by commodity",k,"=",OptimalFlow[p,k])
    return Pi_ij,W,Sigma

In [34]:
Pi_ijWithArcNumber,W,Sigma = PathModel()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-01
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 128 rows, 25 columns and 55 nonzeros
Model fingerprint: 0xb6c5b0b1
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [3e+00, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 128 rows and 25 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.4196000e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.419600000e+04
Flow on path 0 by commodity 1 = 1.0
Flow on path 1 by commodity 2 = 1.0
Flow on path 2 by commodity 3 = 1.0
Flow on path 3 by commodity 4 = 1.0
Flow on path 4 by commodity 5 = 1.0


In [35]:
Pi_ij = {ArcDict[i]:0 for i in range(ArcCountWithDummyArcs)}

for i in range(ArcCountWithDummyArcs):
    Pi_ij[ArcDict[i]] = Pi_ijWithArcNumber[i]

print("Pi=",Pi_ij)
print("W=",W)
print("Sigma=",Sigma)

Pi= {(1, 2): 0.0, (1, 4): 0.0, (1, 6): 0.0, (1, 7): 0.0, (1, 8): 0.0, (1, 10): 0.0, (1, 13): 0.0, (1, 14): 0.0, (1, 16): 0.0, (1, 17): 0.0, (1, 20): 0.0, (2, 4): 0.0, (2, 5): 0.0, (2, 7): 0.0, (2, 8): 0.0, (2, 9): 0.0, (2, 10): 0.0, (2, 14): 0.0, (2, 17): 0.0, (2, 19): 0.0, (3, 5): 0.0, (3, 6): 0.0, (3, 12): 0.0, (3, 13): 0.0, (3, 14): 0.0, (3, 15): 0.0, (3, 17): 0.0, (3, 19): 0.0, (4, 5): 0.0, (4, 8): 0.0, (4, 9): 0.0, (4, 14): 0.0, (4, 19): 0.0, (4, 20): 0.0, (5, 6): 0.0, (5, 11): 0.0, (5, 16): 0.0, (5, 18): 0.0, (5, 19): 0.0, (6, 7): 0.0, (6, 8): 0.0, (6, 9): 0.0, (6, 10): 0.0, (6, 16): 0.0, (6, 17): 0.0, (7, 8): 0.0, (7, 12): 0.0, (7, 17): 0.0, (8, 9): 0.0, (8, 11): 0.0, (8, 12): 0.0, (8, 13): 0.0, (8, 15): 0.0, (8, 16): 0.0, (8, 19): 0.0, (8, 20): 0.0, (9, 10): 0.0, (9, 11): 0.0, (9, 12): 0.0, (9, 14): 0.0, (9, 15): 0.0, (9, 17): 0.0, (9, 18): 0.0, (9, 19): 0.0, (10, 11): 0.0, (10, 12): 0.0, (10, 13): 0.0, (10, 14): 0.0, (10, 16): 0.0, (10, 18): 0.0, (10, 19): 0.0, (11, 17): 0.0, 

In [36]:
ReversedArcDict = {ArcDict[i]:i for i in range(ArcCountWithDummyArcs)}
ReversedArcDict

{(1, 2): 0,
 (1, 4): 1,
 (1, 6): 2,
 (1, 7): 3,
 (1, 8): 4,
 (1, 10): 5,
 (1, 13): 6,
 (1, 14): 7,
 (1, 16): 8,
 (1, 17): 9,
 (1, 20): 10,
 (2, 4): 11,
 (2, 5): 12,
 (2, 7): 13,
 (2, 8): 14,
 (2, 9): 15,
 (2, 10): 16,
 (2, 14): 17,
 (2, 17): 18,
 (2, 19): 19,
 (3, 5): 20,
 (3, 6): 21,
 (3, 12): 22,
 (3, 13): 23,
 (3, 14): 24,
 (3, 15): 25,
 (3, 17): 26,
 (3, 19): 27,
 (4, 5): 28,
 (4, 8): 29,
 (4, 9): 30,
 (4, 14): 31,
 (4, 19): 32,
 (4, 20): 33,
 (5, 6): 34,
 (5, 11): 35,
 (5, 16): 36,
 (5, 18): 37,
 (5, 19): 38,
 (6, 7): 39,
 (6, 8): 40,
 (6, 9): 41,
 (6, 10): 42,
 (6, 16): 43,
 (6, 17): 44,
 (7, 8): 45,
 (7, 12): 46,
 (7, 17): 47,
 (8, 9): 48,
 (8, 11): 49,
 (8, 12): 50,
 (8, 13): 51,
 (8, 15): 52,
 (8, 16): 53,
 (8, 19): 54,
 (8, 20): 55,
 (9, 10): 56,
 (9, 11): 57,
 (9, 12): 58,
 (9, 14): 59,
 (9, 15): 60,
 (9, 17): 61,
 (9, 18): 62,
 (9, 19): 63,
 (10, 11): 64,
 (10, 12): 65,
 (10, 13): 66,
 (10, 14): 67,
 (10, 16): 68,
 (10, 18): 69,
 (10, 19): 70,
 (11, 17): 71,
 (11, 18): 72,


In [37]:
def ReducedCostCompute():
    ReducedCost = {k:[] for k in Commodities}
    
    for k in Commodities:
        ReducedCost[k] = [0]*ArcCountWithDummyArcs
    
    for k in Commodities:
        for i in range(ArcCountWithDummyArcs):
            for p in Paths:
                ReducedCost[k][i] += B_k_Dict[k]*(CostDict[i]-Pi_ijWithArcNumber[i])*Delta_ij_p[p][i] - W*InfoDict[ArcDict[i][1]] #- Sigma[k]
            
    return ReducedCost

In [38]:
def BellmanFord(src,k,ReducedCost):

    # Step 1: Initialize distances from src to all other vertices
    # as INFINITE
    dist = [float("Inf")] * NodeCount
    prev = [-1] * NodeCount
    dist[src-1] = 0
    prev[src-1] = 0

    # Step 2: Relax all edges |V| - 1 times. A simple shortest
    # path from src to any other vertex can have at-most |V| - 1
    # edges
    for _ in range(NodeCount - 1):
        # Update dist value and parent index of the adjacent vertices of
        # the picked vertex. Consider only those vertices which are still in
        # queue
        for u in N:
            for v in NodeNodeAdjacencyList[u]:
#                 print(u,v,k,ReversedArcDict[(u,v)])
                if dist[u-1] != float("Inf") and dist[u-1] + ReducedCost[k][ReversedArcDict[(u,v)]] < dist[v-1]:
                    dist[v-1] = dist[u-1] + ReducedCost[k][ReversedArcDict[(u,v)]]
                    prev[v-1] = u-1

    # Step 3: check for negative-weight cycles. The above step
    # guarantees shortest distances if graph doesn't contain
    # negative weight cycle. If we get a shorter path, then there
    # is a cycle.

    for u in N:
        for v in NodeNodeAdjacencyList[u]:
            if dist[u-1] != float("Inf") and dist[u-1] + ReducedCost[k][ReversedArcDict[(u,v)]] < dist[v-1]:
                print("Graph contains negative weight cycle")
                return

    return dist, prev;
    

In [39]:
# Distance = [0]*NodeCount
# Prev = [0]*NodeCount


In [40]:
# Distance, Previous = BellmanFord(Origin[0],1)
# Destination

In [41]:
def MinReducedCostPath():

    ReducedCost = ReducedCostCompute() 
#     print(ReducedCost)
    SP = [0]*CommodityCount

    for k in Commodities:

        Distance, Previous = BellmanFord(Origin[k-1],k,ReducedCost)
        for i in range(len(Distance)):
            Distance[i] -= Sigma[k]

        if Distance[Destination[k-1]-1] < 0:

            SPTemp = [Destination[k-1]]
            T = Destination[k-1]-1

            while T!=Origin[k-1]-1:

                T = Previous[T]
                SPTemp.insert(0,T+1)

            SP[k-1] = SPTemp

    return SP

In [42]:
def Pricing():
    ShortestPath = MinReducedCostPath()
    print(ShortestPath)

    NewPathCtr = 0
    PathLength = len(Paths)
    for k in range(CommodityCount):
        if ShortestPath[k] != 0:

            NewPathCtr += 1
            Paths.append(PathLength-1+NewPathCtr)
            PathSetForCommodity[k+1].append(PathLength-1+NewPathCtr)
            NodesInPath[PathLength-1+NewPathCtr] = ShortestPath[k]
            PathDict[PathLength-1+NewPathCtr] = []
            PathCost[PathLength-1+NewPathCtr] = 0
            PathInfo[PathLength-1+NewPathCtr] = 0
            Delta_ij_p[PathLength-1+NewPathCtr] = [0]*ArcCountWithDummyArcs

            for i in range(len(ShortestPath[k])-1):
                PathDict[PathLength-1+NewPathCtr].append((ShortestPath[k][i],ShortestPath[k][i+1]))
                PathCost[PathLength-1+NewPathCtr] += CostDict[ReversedArcDict[(ShortestPath[k][i],ShortestPath[k][i+1])]]

            for i in range(len(ShortestPath[k])):
                PathInfo[PathLength-1+NewPathCtr] += InfoDict[ShortestPath[k][i]]

            for i in ArcDict:
                    if ArcDict[i] in PathDict[PathLength-1+NewPathCtr]:
                        Delta_ij_p[p][i] = 1
    return NewPathCtr

In [43]:
NewPathCtr = Pricing()
IterationCounter = 1
while NewPathCtr != 0:
    IterationCounter += 1
    Pi_ijWithArcNumber,W,Sigma = PathModel()
#     print(Pi_ijWithArcNumber)
#     print(W)
#     print(Sigma)
    NewPathCtr = Pricing()
#     print(NewPathCtr)
    


[[2, 8, 13], [7, 8, 20], [2, 5, 6, 17], [1, 2, 5, 11], [7, 8, 19]]
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 148 rows, 50 columns and 115 nonzeros
Model fingerprint: 0x4fe480a1
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [3e+00, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 148 rows and 50 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.800000000e+02
Flow on path 5 by commodity 1 = 1.0
Flow on path 6 by commodity 2 = 1.0
Flow on path 2 by commodity 3 = 1.0
Flow on path 8 by commodity 4 = 1.0
Flow on path 9 by commodity 5 = 1.0
[[2, 10, 13], [7, 12, 20], [2, 7, 17], [1, 8, 